## Dynamics
We consider the control of a car whose state $z(t)$ (encapsulating position, velocity, etc of the car) at time $t$ is controlled by a control variable $u(t)$ (encapsulating acceleration, steering angle, etc) through dynamics of the form

$$ \dot z(t) = \psi(z(t), u(t)),$$

where $\psi$ models the physics of the movement, as described below.

The continuous-time representation of the car is not amenable to numerical computations. We consider the discrete-time
representation of the movement given by some discretization of the continuous-time dynamics, which lead to a sequence of
states $z_0, z_1, \ldots $, called the trajectory, whose discrete-time dynamics are of the form

$$ z_{t+1} = \phi(z_t, u_t), $$

for $u_0, u_1, \ldots$ a sequence of controls applied to the system.

The simplest discretization method is the Euler method, which for a time-step $dt$, reads $z_{t+1} = z_t + \Delta \psi(z_t, u_t),$
i.e., $\phi(z_t, u_t) = z_t +\Delta \psi(z_t, u_t)$ where $\Delta$ is the discretization step. The Euler method is illustrated in the case of the simple model of a car. Alternatively, we can consider a Runge Kutta of order 4 (see, e.g. [[Süli and Mayers, 2003]](#refs)), which is illustrated in the realistic model of a car.

In [1]:
import torch
import time
import math
import pandas as pd
import os
from matplotlib import pyplot as plt

<a id='simple_model'></a>
### Simple model of a car
A simple model of a car is illustrated below.


<center><img src="fig/simple_model.png" alt="simple_model" width="400" /></center>

The state of the car is described by $z(t) = (x(t), y(t), \theta(t), v(t))$, where

- the position $(x, y)$ of the car on the plane,
- the yaw angle $\theta$, i.e., the angle between the orientation of the car and the x-axis,
- the velocity $v$ of the car.

The car is controlled by $u(t) = (a(t), \delta(t))$, where
- its acceleration $a$,
- the steering angle $\delta$.

The dynamics are, dropping the dependency w.r.t. time,
\begin{align}
\dot x &  = v \cos(\theta)  & \dot \theta &  = v \tan(\delta)/L  \\
\dot y  & = v \sin(\theta)  & \dot v&  =  a,
\end{align}
where $L$ is the length of the car.

The following code instantiates this simple model of a car with an Euler discretization. The rendering is inspired by the rendering made in Open AI gym.

In [2]:
from envs.car import set_window_viewer, add_car_to_viewer

class SimpleCar:
    def __init__(self, dt=0.02, vinit=1., constrain_angle=math.pi/3):
        super(SimpleCar, self).__init__()
        self.dim_state, self.dim_ctrl = 4, 2
        self.dt = dt
        
        self.constrain_angle = constrain_angle
        
        # Initial state of the system
        self.init_state = self.state = torch.tensor([0, 0, 0., vinit])

        # Dimensions of the car
        self.carlength, self.carwidth = 0.09, 0.045

        self.viewer = self.cartrans = None

    # Continuous dynamics as defined above
    def dyn(self, state, ctrl):
        x, y, theta, v = state
        a, delta = ctrl
        
        # We constrain the steering angle to be bounded through some non-linear transformation
        delta = 2/math.pi*torch.arctan(delta)*self.constrain_angle
        dx = v * torch.cos(theta)
        dy = v * torch.sin(theta)
        dtheta = v * torch.tan(delta)/self.carlength
        dv = a
        return torch.tensor([dx, dy, dtheta, dv])

    # Discrete dynamics
    def discrete_dyn(self, state, ctrl):
        # Euler method
        return state + self.dt * self.dyn(state, ctrl)

    # Let the system make a step with the given control
    def step(self, ctrl):
        next_state = self.discrete_dyn(self.state, ctrl)
        self.state = next_state
        return next_state

    def roll_out_cmd(self, cmd, reset=True):
        if reset:
            self.reset()
        state = self.state
        traj = [state]
        for ctrl in cmd:
            state = self.discrete_dyn(state, ctrl)
            traj.append(state)
        return traj
    
    def reset(self):
        self.state = self.init_state
   
    def set_viewer(self, traj):
        stack_traj = torch.stack(traj)
        min_x, max_x = min(stack_traj[:, 0]), max(stack_traj[:, 0])
        min_y, max_y = min(stack_traj[:, 1]), max(stack_traj[:, 1])
        self.viewer = set_window_viewer(min_x, max_x, min_y, max_y, 2*self.carlength)
        self.cartrans = add_car_to_viewer(self.viewer, self.carlength, self.carwidth)
        
    def render(self):
        if self.viewer is None:
            raise ValueError('Initilaze the viewer before rendering')
        x, y, theta = self.state.numpy()[:3]   
        self.cartrans.set_translation(x, y) # Translates the car with the new position      
        self.cartrans.set_rotation(theta) # Update the orientation of the car
        return self.viewer.render()

We can now test the movement of our car. You can select a sequence of steering angles (``deltas``) and a sequence of accelerations (``accs``) in the code below to see the movement of the car. 

In [3]:
dt = 0.02 # Discretization step
horizon = 400 # Number of steps
car = SimpleCar(dt)

deltas = torch.tan(math.pi/6  * torch.ones(horizon)) # Constant steering angle
accs = torch.linspace(0, 1, horizon) # Linearly increasing acceleration
cmd = torch.stack((accs, deltas), dim=1) # Sequence of controls

# Display the movement of the car
car.reset()
traj = car.roll_out_cmd(cmd)
car.set_viewer(traj)
car.render()
for ctrl in cmd:
    time.sleep(dt)
    car.step(ctrl)
    car.render()
car.viewer.close()

2024-08-01 16:04:48.979 Python[59711:1687881] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/pf/znths45963l2918jylv3wzjr0134jq/T/org.python.python.savedState


<a id='realistic_model'></a>

### Realistic model of a car
A realistic model of a car considers the friction forces on the wheels to define how the car turns.  Specifically, we consider a bicycle model as presented by [[Liniger et al 2015]](#refs) to model a miniature car or [[Gerdts et al 2009]](#refs) to model a Volkswagen car. The model is illustrated in the figure below.

<center><img src="fig/realistic_model.png" alt="realistic model" width="600" /></center>

In this model, the state of the car is defined by $z(t) = (x(t), y(t), \theta(t), v_x(t), v_y(t), \omega(t))$, where
- the position $(x, y)$ of the car,
- the yaw of the car $\theta$
- the longitudinal speed $v_x$,
- the lateral speed $v_y$,
- the yaw rate $\omega$.

The controls are given by $u(t) = (a(t), \delta(t))$, where
- the PWM duty cycle of the car denoted $a$, this duty cycle can be negative to take into account braking,
- the steering angle $\delta$.

These controls act on the state through the following forces.
- A longitudinal force on the rear wheels, denoted $F_{r, x}$ modeled using a motor model for the DC electric motor as well as a friction model for the rolling resistance and the drag:
$$
F_{r, x} = (C_{m1} -C_{m2} v_x) a - C_{r0} - C_{r2} v_x^2,
$$
where $C_{m1}, C_{m2}, C_{r0}, C_{r2}$ are constants estimated from experiments. 
- Lateral forces on the front and rear wheels, denoted $F_{f, y}, F_{y, r}$ respectively, modeled using a simplified Pacejka Tire Model: 
\begin{align*}
F_{f, y} & = D_f \sin(C_f \arctan(B_f \alpha_f)) \quad \textrm{where} \ \alpha_f = \delta - \arctan\left(\frac{\omega l_f + v_y}{v_x}\right)  \\
F_{r, y} & = D_r \sin(C_r \arctan(B_r \alpha_r)) \quad \textrm{where} \ \alpha_r = \arctan\left(\frac{\omega l_r - v_y}{v_x}\right),  
\end{align*}
where $\alpha_f$, $\alpha_r$ are the slip angles on the front and rear wheels respectively, $l_f, l_r$ are the distance from the center of gravity to the front and the rear wheel and the constants $B_r, C_r, D_r, B_f, C_f, D_f$ define the exact shape of the semi-empirical curve, presented below

<center><img src="fig/pacejka.png" alt="pacejka tire model" width="350" /></center>

The continuous time dynamics are then (dropping the dependency w.r.t. time for simplicity):

\begin{align*}
\dot x & = v_x \cos(\theta) - v_y\sin(\theta) & \dot v_x & =\frac{1}{m}(F_{r,x} - F_{f, y} \sin (\delta)) + v_y \omega \\
\dot y & = v_x\sin(\theta) + v_y\cos(\theta) & \dot v_y & = \frac{1}{m}(F_{r, y} + F_{f, y} \cos(\delta)) - v_x \omega \\
\dot \theta & = \omega & \dot \omega & = \frac{1}{I_z}(F_{f, y} l_f \cos(\delta) - F_{r, y} l_r),
\end{align*}

where $m$ is the mass of the car and $I_z$ is the inertia.

The following code illustrates these dynamics. The parameters used in this model are the ones of [[Liniger et al 2015]](#refs). 

In [4]:
from envs.car import set_window_viewer, add_car_to_viewer

class RealisticCar:
    def __init__(self, dt=0.02, vinit=1., constrain_angle=math.pi/3, acc_bounds=(-0.1, 1.)):
        super(RealisticCar, self).__init__()
        self.dim_state, self.dim_ctrl = 6, 2
        self.dt = dt
        self.constrain_angle, self.acc_bounds = constrain_angle, acc_bounds
        
        # Initial state of the system
        self.init_state = torch.tensor([0, 0, 0, vinit, 0., 0.])
        
        # Constants used to define the dynamics are stored in a json file
        self.dyn_csts = pd.read_json(os.path.join('envs/bicycle_model.json'), typ='series')
        
        # We use larger carlength and carwidth than the parameters of Liniger et al for better visualization
        self.carlength, self.carwidth = 1.5 * self.dyn_csts['car_l'], 1.5 * self.dyn_csts['car_w']

        self.viewer = self.cartrans = None

    # Continuous dynamics as defined above
    def dyn(self, state, ctrl):
        x, y, theta, vx, vy, vtheta = state
        a, delta = ctrl
        
        # We constrain the steering angle to be bounded through some non-linear transformation
        delta = 2/math.pi*torch.arctan(delta)*self.constrain_angle
        
        # Similarly we constrain the acceleration to some bounds taken from Liniger et al 
        gap = self.acc_bounds[1] - self.acc_bounds[0]
        a = gap*torch.sigmoid(4 * a / gap) + self.acc_bounds[0]
        
        Cm1, Cm2, Cr0, Cr2,\
        Br, Cr, Dr, Bf, Cf, Df,\
        m, Iz, lf, lr = [self.dyn_csts[key] for key in ['Cm1', 'Cm2', 'Cr0', 'Cr2',
                                                        'Br', 'Cr', 'Dr', 'Bf', 'Cf', 'Df',
                                                        'm', 'Iz', 'lf', 'lr']]

        # Note that the angles alphaf and alphar must be computed using arctan2
        alphaf = delta - torch.atan2(vtheta*lf + vy, vx) 
        alphar = torch.atan2(vtheta*lr - vy, vx)
        Fry = Dr*torch.sin(Cr*torch.atan(Br*alphar))
        Ffy = Df*torch.sin(Cf*torch.atan(Bf*alphaf))
        Frx = (Cm1 - Cm2*vx)*a - Cr0 - Cr2*vx**2
        dx = torch.cos(theta)*vx - torch.sin(theta)*vy
        dy = torch.sin(theta)*vx + torch.cos(theta)*vy
        dtheta = vtheta
        dvx = (Frx - Ffy*torch.sin(delta) + m*vy*vtheta)/m
        dvy = (Fry + Ffy*torch.cos(delta) - m*vx*vtheta)/m
        dvtheta = (Ffy*lf*torch.cos(delta) - Fry*lr)/Iz
        return torch.stack([dx, dy, dtheta, dvx, dvy, dvtheta])

    # Discrete dynamics
    def discrete_dyn(self, state, ctrl):
        # Runge Kutta method of order 4
        # We consider a constant control variable in this discretization
        dt = self.dt
        k1 = self.dyn(state, ctrl)
        k2 = self.dyn(state + dt*k1/2, ctrl)
        k3 = self.dyn(state + dt*k2/2, ctrl)
        k4 = self.dyn(state + dt*k3, ctrl)
        return state + dt*(k1 + 2*k2 + 2*k3 + k4)/6

    # Let the system make a step with the given control
    def step(self, ctrl):
        next_state = self.discrete_dyn(self.state, ctrl)
        self.state = next_state
        return next_state

    def roll_out_cmd(self, cmd, reset=True):
        if reset:
            self.reset()
        state = self.state
        traj = [state]
        for ctrl in cmd:
            state = self.discrete_dyn(state, ctrl)
            traj.append(state)
        return traj

    def reset(self):
        self.state = self.init_state
        
    def set_viewer(self, traj):
        stack_traj = torch.stack(traj)
        min_x, max_x = min(stack_traj[:, 0]), max(stack_traj[:, 0])
        min_y, max_y = min(stack_traj[:, 1]), max(stack_traj[:, 1])
        self.viewer = set_window_viewer(min_x, max_x, min_y, max_y, 2*self.carlength)
        self.cartrans = add_car_to_viewer(self.viewer, self.carlength, self.carwidth)
        
    def render(self):
        if self.viewer is None:
            raise ValueError('Initiliaze the viewer before rendering')
        x, y, theta = self.state.numpy()[:3]   
        self.cartrans.set_translation(x, y) # Translates the car with the new position      
        self.cartrans.set_rotation(theta) # Update the orientation of the car
        return self.viewer.render()

As for the simple model of the car, we can test the movement of the car for a sequence of steering angles (``deltas``) and a sequence of accelerations (``accs``). Note the difference of behavior in the case of a constant steering angle and a linearly increasing acceleration: in the case of a realistic car, the trajectory is not a simple circle, and we see the car drift. Note also that the model is not realistic for low speeds, therefore we initialize the car with a constant longitudinal speed (``vinit`` in the definition of the model) 

In [5]:
dt = 0.02 # Discretization step
horizon = 400 # Number of steps

car = RealisticCar(dt)

# To better observe the drifting of the car, change the bounds on the acceleration by uncommenting the following
# car = RealisticCar(dt, acc_bounds=(-2., 2.))

accs = torch.linspace(0, 2, horizon) # Linearly increasing acceleration
deltas = torch.tan(math.pi/6*torch.ones(horizon)) # Constant steering angle
cmd = torch.stack((accs, deltas), dim=1) # Sequence of controls

traj = car.roll_out_cmd(cmd)
car.set_viewer(traj)
car.render()
for ctrl in cmd:
    time.sleep(dt)
    car.step(ctrl)
    car.render()
car.viewer.close()

Now that we have the dynamics we need to define the costs, see the notebook [Costs](costs.ipynb).

<a id='refs'></a>
### References
**Optimization‐based autonomous racing of 1: 43 scale RC cars.**  
Alexander Liniger, Alexander Domahidi, and Manfred Morari.  
*Optimal Control Applications and Methods, 2015.*  

**Generating Locally Optimal Trajectories for an Automatically Driven Car.**  
Matthias Gerdts, Simon Karrenberg, Bernhard Müller-Beßler and Gregor Stock.  
*Optimization and Engineering, 2009.*  

**An Introduction to Numerical Analysis.**  
Endre Süli and David Mayers.  
*Cambridge University Press, 2003.*  